<a href="https://colab.research.google.com/github/ekaterga/stc510/blob/main/Module5Essentials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
with open('jeopardy.json', 'r') as file:
  data_dict = json.load(file)

In [4]:
data = pd.DataFrame(data_dict)

In [5]:
data['value'] = data['value'].astype(str).replace('[\$,]', '', regex=True)
data['value'] = pd.to_numeric(data['value'], errors='coerce').fillna(0).astype(int)

In [6]:
data['value_class'] = data['value'].apply(lambda x: 'high' if x >= 800 else 'low')

Taking out columns that are not relevant to the project

In [7]:
cleaned_data = data.drop(columns=['category', 'air_date', 'round', 'show_number', 'value'])

In [8]:
print(cleaned_data)

                                                 question  \
0       'For the last 8 years of his life, Galileo was...   
1       'No. 2: 1912 Olympian; football star at Carlis...   
2       'The city of Yuma in this state has a record a...   
3       'In 1963, live on "The Art Linkletter Show", t...   
4       'Signer of the Dec. of Indep., framer of the C...   
...                                                   ...   
216925  'This Puccini opera turns on the solution to 3...   
216926  'In North America this term is properly applie...   
216927  'In Penny Lane, where this "Hellraiser" grew u...   
216928  'From Ft. Sill, Okla. he made the plea, Arizon...   
216929  'A silent movie title includes the last name o...   

                                answer value_class  
0                           Copernicus         low  
1                           Jim Thorpe         low  
2                              Arizona         low  
3                          McDonald\'s         low  
4 

I decided that for the essentials project I was going to convert the data into TFIDF vectors and then train a SVM classifier to predict the question values. I was about 4 hours in to letting the SVM classifier train and my runtime disconnected so I decided to pivot and only look at a fraction of the dataset. Only using half of it took about an hour. If you would like me to redo the assignment I can also just let it run but I was worried since it got stuck. When troubleshooting it was suggested that there was mssing data which I looked at further down in the code but discovered there is no missing data.

In [9]:
subset_data = cleaned_data.sample(frac=0.50, random_state=42)

In [11]:
X_subset = subset_data['question']
y_subset = subset_data['value_class']

In [13]:
tfidf_vectorizer = TfidfVectorizer()
X_tfidf_subset = tfidf_vectorizer.fit_transform(X_subset)

In [14]:
X_train_subset, X_test_subset, y_train_subset, y_test_subset = train_test_split(X_tfidf_subset, y_subset, test_size=0.2, random_state=42)

In [15]:
svm_classifier_subset = SVC()
svm_classifier_subset.fit(X_train_subset, y_train_subset)

SVC()

In [16]:
y_pred_subset = svm_classifier_subset.predict(X_test_subset)

In [17]:
accuracy_subset = accuracy_score(y_test_subset, y_pred_subset)
print("Accuracy on subset:", accuracy_subset)

Accuracy on subset: 0.5897294057991057


In [18]:
print("Missing values:", cleaned_data.isnull().sum())

Missing values: question       0
answer         0
value_class    0
dtype: int64
